In [1]:
import pandas as pd
import backtesting as bt
import pandas_ta as ta
import numpy as np
import datetime as dt
from IPython.display import clear_output

In [17]:
filename = 'data/NESIND/NESIND_5minute_90days.csv'
data = pd.read_csv(filename)
data.set_index('datetime',inplace=True)
data

,Close,exchange_code,High,Low,Open,stock_code,Volume
datetime,,,,,,,
2024-01-29 09:05:00,2482.15,NSE,2482.15,2482.15,2482.15,NESIND,0
2024-01-29 09:15:00,2482.00,NSE,2488.90,2476.60,2482.15,NESIND,39718
2024-01-29 09:20:00,2497.00,NSE,2497.55,2481.90,2482.00,NESIND,22338
2024-01-29 09:25:00,2500.75,NSE,2505.00,2496.50,2496.95,NESIND,16544
2024-01-29 09:30:00,2499.25,NSE,2502.20,2499.25,2500.25,NESIND,8660
...,...,...,...,...,...,...,...
2024-04-26 15:05:00,2479.50,NSE,2479.95,2473.00,2477.40,NESIND,37317
2024-04-26 15:10:00,2481.60,NSE,2482.80,2479.55,2479.55,NESIND,30752
2024-04-26 15:15:00,2482.90,NSE,2485.70,2479.65,2481.65,NESIND,39890


In [23]:
data.reset_index(inplace=True)

In [31]:
class rsi(bt.Strategy):

    target = 10
    rsi_length = 14
    rsi_upperband = 70
    rsi_lowerband = 30



    def init(self):
        self.rsi = self.I(ta.rsi,self.data.Close.s,self.rsi_length)

    
    def next(self):
        
        price = self.data.Close[-1]
        current_rsi = self.rsi[-1]
        


        target= self.target/10
        target_precent = target/100
        target_price = (1-target_precent)*price


        

        if self.rsi_upperband <= current_rsi:
            if not self.position.is_short:
                self.sell(size = 200,tp=target_price,sl=1.01*price)
        
    

In [32]:
eng = bt.Backtest(data,rsi,cash=4000000,commission=0.002)

C:\Users\himan\AppData\Local\Temp\ipykernel_6708\188491853.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  eng = bt.Backtest(data,rsi,cash=4000000,commission=0.002)


In [33]:
stats = eng.run()
stats

Start                                     0.0
End                                    4590.0
Duration                               4590.0
Exposure Time [%]                    43.75953
Equity Final [$]                   3983402.24
Equity Peak [$]                    4013877.44
Return [%]                          -0.414944
Buy & Hold Return [%]                0.517696
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.222931
Avg. Drawdown [%]                   -0.131644
Max. Drawdown Duration                 4041.0
Avg. Drawdown Duration                  284.8
# Trades                                 37.0
Win Rate [%]                        56.756757
Best Trade [%]                       0.940543
Worst Trade [%]                     -1.389335
Avg. Trade [%]                    

In [34]:
eng.plot()

c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2027', ...)

In [37]:
opt_stats,heatmap = eng.optimize(
    target = range(2,20),
    rsi_length = range(5,25),
    maximize = 'Win Rate [%]',
    return_heatmap = True

)
stats

c:\Users\himan\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 360 configurations.
  output = _optimize_grid()


ValueError: Short orders require: TP (2487.8144) < LIMIT (2487.8144) < SL (2517.728)